In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import sklearn

import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff
import pandas_profiling as pp

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, classification_report, roc_curve,precision_recall_curve, auc,confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.impute import KNNImputer

from xgboost import XGBClassifier

from catboost import CatBoostClassifier

In [ ]:
df = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
df.head()

In [ ]:
df = df.drop('id', axis=1)
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df['bmi'].fillna(df['bmi'].mean(), inplace = True) #Filled empty bmi with mean value

In [ ]:
df.isnull().sum()

In [ ]:
labels =df['stroke'].value_counts(sort = True).index
sizes = df['stroke'].value_counts(sort = True)

colors = ["lightblue","red"]
explode = (0.05,0) 
 
plt.figure(figsize=(7,7))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=90,)

plt.title('Number of stroke in the dataset')
plt.show()

In [ ]:
#****************
en_df = df.copy()
en_df.head()

In [ ]:
df.head()

In [ ]:
smoke_to_int = {
    'never smoked': 0,
    'formerly smoked': 1,
    'smokes': 2,
    'Unknown': -1
}
en_df['smoking_status'] = [smoke_to_int[s] for s in en_df['smoking_status']]
en_df.head()


In [ ]:
work_to_int = {
    'Private': 1,
    'Self-employed': 2,
    'Govt_job': 3,
    'children': 4,
    'Never_worked': 0
}
en_df['work_type'] = [work_to_int[s] for s in en_df['work_type']]
en_df.head()

In [ ]:
en_df['gender'] = [int(m) for m in en_df['gender'] == 'Female']
en_df['Residence_type'] = [int(r) for r in en_df['Residence_type'] == 'Urban']
en_df['ever_married'] = [int(b) for b in en_df['ever_married'] == 'Yes']
en_df.head()

In [ ]:
df.head()

In [ ]:
pp.ProfileReport(en_df)

In [ ]:
def plot_hist(col, bins=30, title="",xlabel="",ax=None):
    sns.distplot(col, bins=bins,ax=ax)
    ax.set_title(f'Histogram of {title}',fontsize=20)
    ax.set_xlabel(xlabel)
    


In [ ]:
fig, axes = plt.subplots(1,3,figsize=(11,7),constrained_layout=True)
plot_hist(df.bmi,
          title='Bmi',
          xlabel="Level of the BMI",
          ax=axes[0])
plot_hist(df.age,
          bins=30,
          title='Age',
          xlabel='Age',
          ax=axes[1])
plot_hist(df.avg_glucose_level,
          title='Serum Creatinine', 
          xlabel='Level of serum creatinine in the blood (mg/dL)',
          ax=axes[2])

plt.show()

In [ ]:
sns.catplot(y="work_type", hue="stroke", kind="count",
            palette="pastel", edgecolor=".6",
            data=df)

In [ ]:
sns.catplot(y="smoking_status", hue="stroke", kind="count",
            palette="pastel", edgecolor=".6",
            data=df)

In [ ]:
plt.figure(figsize=(17,7))
sns.catplot(x="gender", y="stroke", hue="heart_disease", palette="pastel", kind="bar", data=df)
sns.catplot(x="gender", y="stroke", hue="Residence_type", palette="pastel", kind="bar", data=df)
sns.catplot(x="gender", y="stroke", hue="hypertension", palette="pastel", kind="bar", data=df)
plt.show()

In [ ]:
len_data = len(df)
len_w = len(df[df["gender"]=="Male"])
len_m = len_data - len_w

men_stroke = len(df.loc[(df["stroke"]==1)&(df['gender']=="Male")])
men_no_stroke = len_m - men_stroke

women_stroke = len(df.loc[(df["stroke"]==1) & (df['gender']=="Female")])
women_no_stroke = len_w - women_stroke

labels = ['Men with stroke','Men healthy','Women with stroke','Women healthy']
values = [men_stroke, men_no_stroke, women_stroke, women_no_stroke]

fig = go.Figure(data=[go.Pie(labels=labels, values=values,textinfo='label+percent',hole=0.4)])
fig.update_layout(
    title_text="Distribution of stroke EVENT according to their gender")
fig.show()

In [ ]:
df.columns

In [ ]:
features=['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type',
       'smoking_status']
from matplotlib.offsetbox import AnchoredText
correlation_table = []
for cols in features:
    y = en_df["stroke"]
    x = en_df[cols]
    corr = np.corrcoef(x, y)[1][0]
    dict ={
        'Features': cols,
        'Correlation coefficient' : corr,
        'Feat_type': 'numerical'
    }
    correlation_table.append(dict)
dF1 = pd.DataFrame(correlation_table)
fig = plt.figure(figsize=(10,6), facecolor='#EAECEE')
ax = sns.barplot(x="Correlation coefficient", y="Features", 
                     data=dF1.sort_values("Correlation coefficient", ascending=False),
                     palette='viridis', alpha=0.75)
ax.grid()
#ax.set_title("Correlation of numerical features with Target", fontsize=20, y=1.05)

title =  'Correlation features with target'
sub_title = 'In comparison with categorical features \
\nnumericals are less correlated with target.'

plt.gcf().text(0.05, 1.02, title, fontsize=24)
#plt.gcf().text(0.05, 0.9, sub_title, fontsize=14)

at1 = AnchoredText(sub_title,
                   loc='lower left', frameon=True,
                   bbox_to_anchor=(-0.1, 1.01),
                   bbox_transform=ax.transAxes,
                   #prop=dict(size=8),
                   )
at1.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
ax.add_artist(at1)

In [ ]:
plt.figure(figsize=(16,8))
sns.heatmap(en_df.corr(),cmap="Blues");

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

X = en_df[features]
y = en_df['stroke']
forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)
forest.fit(X, y)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

    
    
# Plot the impurity-based feature importances of the forest
plt.figure()

plt.title("Feature importances")
sns.barplot(x=np.array(features)[indices], y=importances[indices], palette="deep",yerr=std[indices])
plt.xticks(range(X.shape[1]), np.array(features)[indices],rotation=60)
plt.xlim([-1, X.shape[1]])
plt.show()

In [ ]:
#en_df_imputed = en_df
#imputer = KNNImputer(n_neighbors=4, weights="uniform")
#imputer.fit_transform(en_df_imputed)

In [ ]:
#en_df_imputed.isnull().sum()

In [ ]:
df.head()

In [ ]:
#en_df_imputed.head()

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

models_score = []

def plot_cm(cm,title):
    z = cm
    x = ['No stroke', 'stroke']
    y = x
    # change each element of z to type string for annotations
    z_text = [[str(y) for y in x] for x in z]

    # set up figure 
    fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='deep')

    # add title
    fig.update_layout(title_text='<i><b>Confusion matrix {}</b></i>'.format(title),
                      #xaxis = dict(title='x'),
                      #yaxis = dict(title='x')
                     )

    # add custom xaxis title
    fig.add_annotation({'font':{'color':"black",'size':14},
                            'x':0.5,
                            'y':-0.10,
                            'showarrow':False,
                            'text':"Predicted value",
                            'xref':"paper",
                            'yref':"paper"})
    
    fig.add_annotation({'font':{'color':"black",'size':14},
                            'x':-0.15,
                            'y':0.5,
                            'showarrow':False,
                            'text':"Real value",
                            'textangle':-90,
                            'xref':"paper",
                            'yref':"paper"})


    # adjust margins to make room for yaxis title
    fig.update_layout(margin={'t':50, 'l':20},width=750,height=750)
    


    # add colorbar
    fig['data'][0]['showscale'] = True
    fig.show()

    

X , y = en_df[features],en_df["stroke"]
#data split to Training Set and Test Set
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)



# RF_model = RandomForestClassifier(n_estimators=50, criterion='entropy') # create object model
# RF_model.fit(x_train,y_train) # fit the model
# # model score
# predict_train_RF = RF_model.predict(x_train)
# predict_test_RF = RF_model.predict(x_test)
#  # accuracy score
# RF_test_score = RF_model.score(x_test,y_test) 
      
# print('------------------------')
# print("Random Forest model before SMOTE")
# print('the score is: ',RF_test_score, ' critetion:', 'entropy', ' num estimators:', 50)
# ----------------------

sm = SMOTE()
x_smote, y_smote = sm.fit_resample(x_train,y_train)


#After SMOTE 
labels = df['stroke'].value_counts(sort = True).index
sizes = y_smote.value_counts(sort = True)

colors = ["lightblue","red"]
explode = (0.05,0) 
 
plt.figure(figsize=(7,7))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=90,)

plt.title('Number of stroke in the dataset after SMOTE')
plt.show()




In [ ]:

best_score = 0
for critetion in ['gini', 'entropy']:
    for i in range(10, 301, 20):
        RF_model = RandomForestClassifier(n_estimators=i, criterion=critetion) # create object model
        RF_model.fit(x_train,y_train) # fit the model
        # model score
        predict_train_RF = RF_model.predict(x_train)
        predict_test_RF = RF_model.predict(x_test)
        # accuracy score
        RF_test_score = RF_model.score(x_test,y_test) 
        print('critetion: ', critetion, ' | num estimators: ',i, ' | Validation Acuuracy: ',RF_test_score)
        if RF_test_score > best_score:
            best_score = RF_test_score
            best_critetion = critetion
            best_estimators = i
print('------------------------')
print("Random Forest model")
print('the BEST score is: ',best_score, ' critetion:', best_critetion, ' num estimators:', best_estimators)
cm_model = confusion_matrix(y_test, predict_test_RF)
print(cm_model)
plot_cm(cm_model,title="Random Forest model") #print Confusion Matrix 

models_score.append(best_score)


In [ ]:
# create object model
SVC_model = SVC()

# fit the model
SVC_model.fit(x_train,y_train)

# model score
predict_train_SVC = SVC_model.predict(x_train)
predict_test_SVC = SVC_model.predict(x_test)

# accuracy score
#SVM_train_score = SVC_model.score(x_train,y_train)
SVM_test_score = SVC_model.score(x_test,y_test)


print("SVC model")
cm_model1 = confusion_matrix(y_test, predict_test_SVC)
print(cm_model1)
print('Validation Acuuracy: ',accuracy_score(y_test, predict_test_SVC))
plot_cm(cm_model1,title="SVC model") #print Confusion Matrix 

models_score.append(SVM_test_score)

In [ ]:
from sklearn.model_selection import train_test_split as tts
import tensorflow as tf
from sklearn.metrics import accuracy_score


In [ ]:
#SPLITTING THE DATA INTO TRAINING AND TESTING DATA
x_train,x_test,y_train,y_test_ann=tts(x,y,test_size=0.2)

In [ ]:
#CREATING ARTIFICIAL NEURAL NETWORK MODEL[ANN]
ann=tf.keras.Sequential()

ann.add(tf.keras.layers.Dense(units=25,activation='relu'))

ann.add(tf.keras.layers.Dense(units=25,activation='relu'))

ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

ann.compile('adam','binary_crossentropy',metrics=['accuracy'])

In [ ]:
# TRAINING ANN MODEL
result=ann.fit(x_train,y_train,epochs=10)

In [ ]:
#CREATING CONFUSION MATRIX FOR THE ACTUAL AND PREDICTED VALUE
from sklearn.metrics import confusion_matrix
y_pred=[]
for i in ann.predict(x_test):
    if i>0.5:
        y_pred.append(1)
    if i<0.5:
        y_pred.append(0)
confusion_matrix(y_test_ann,y_pred)

In [ ]:
# ACCURACY SCORE FOR TESTING DATA
accuracy=accuracy_score(y_test_ann,y_pred)
print('accuracy: ', accuracy)

models_score.append(accuracy)

In [ ]:
def hist_score(score):
    models_names = [
    'Random Forest Classifier',
    'SVM',
    'ANN']

    plt.rcParams['figure.figsize']=20,8
    sns.set_style('darkgrid')
    ax = sns.barplot(x=models_names, y=score, palette = "inferno", saturation =2.0)
    plt.xlabel('Classifier Models', fontsize = 20 )
    plt.ylabel('% of Accuracy', fontsize = 20)
    plt.title('Accuracy of different Classifier Models on test set', fontsize = 20)
    plt.xticks(fontsize = 12, horizontalalignment = 'center', rotation = 8)
    plt.yticks(fontsize = 12)
    for i in ax.patches:
        width, height = i.get_width(), i.get_height()
        x, y = i.get_xy() 
        ax.annotate(f'{round(height,2)}%', (x + width/2, y + height*1.02), ha='center', fontsize = 'x-large')
    plt.show()
    
hist_score(models_score)

In [ ]:
# from imblearn.over_sampling import SMOTE


# X , y = en_df[features],en_df["stroke"] #en_df_imputed[features],en_df_imputed["stroke"]
# x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=23)
# sm = SMOTE()
# X_res, y_res = sm.fit_resample(x_train,y_train)



# def hist_score(score):
#     models_names = [
#     'Random Forest Classifier',
#     'SVM',
#     'ANN']

#     plt.rcParams['figure.figsize']=20,8
#     sns.set_style('darkgrid')
#     ax = sns.barplot(x=models_names, y=score, palette = "inferno", saturation =2.0)
#     plt.xlabel('Classifier Models', fontsize = 20 )
#     plt.ylabel('% of Accuracy', fontsize = 20)
#     plt.title('Accuracy of different Classifier Models on test set', fontsize = 20)
#     plt.xticks(fontsize = 12, horizontalalignment = 'center', rotation = 8)
#     plt.yticks(fontsize = 12)
#     for i in ax.patches:
#         width, height = i.get_width(), i.get_height()
#         x, y = i.get_xy() 
#         ax.annotate(f'{round(height,2)}%', (x + width/2, y + height*1.02), ha='center', fontsize = 'x-large')
#     plt.show()
    

# def run_exp_on_feature(x_train,y_train,x_test,y_test):
#     #x_train,x_test,y_train,y_test = train_test_split(features,labels, test_size=0.2, random_state=23)
#     models= [['Random Forest Classifier ',RandomForestClassifier()],
#             ['SVM ',SVC()]]

#     models_score = []
#     for name,model in models:

#         model = model
#         model.fit(x_train,y_train)
#         model_pred = model.predict(x_test)
#         cm_model = confusion_matrix(y_test, model_pred)
#         #print(cm_model)
#         models_score.append(accuracy_score(y_test,model.predict(x_test)))

#         fpr, tpr, thresholds = roc_curve(y_test, model_pred)
        
#     return models_score


# models_score = run_exp_on_feature(x_train,y_train,x_test,y_test)

# models_score.append(accuracy_score(y_test_ann,y_pred))

# hist_score(models_score)